# <h2 style="text-align:center">Data Encryption Standard </h2>

In [1]:
import random

def encrypt(plaintext, key):
    """Encrypts plaintext using DES with the given key."""
    nblocks = ((len(plaintext) - 1) // 8) + 1
    ciphertext = ''

    for block in range(nblocks):
        block_start = block * 8
        block_end = min(block_start + 8, len(plaintext))
        plaintext_block = plaintext[block_start:block_end]
        ciphertext_block = des(plaintext_block, key)
        ciphertext += ciphertext_block

    return ciphertext

def decrypt(ciphertext, key):
    """Decrypts ciphertext using DES with the given key."""
    nblocks = ((len(ciphertext) - 1) // 8) + 1
    plaintext = ''

    for block in range(nblocks):
        block_start = block * 8
        block_end = min(block_start + 8, len(ciphertext))
        ciphertext_block = ciphertext[block_start:block_end]
        plaintext_block = des(ciphertext_block, key, decrypt=True)
        plaintext += plaintext_block

    return plaintext

def des(block, key, decrypt=False):
    """Encrypts or decrypts a single block using DES with the given key."""
    # Initial permutation
    block = permute(block, IP)

    # Split block into left and right halves
    L = block[:32]
    R = block[32:]

    # Perform 16 rounds
    for i in range(16):
        # Expand right half
        ER = permute(R, E)

        # XOR with round key
        if decrypt:
            KR = key[15 - i]
        else:
            KR = key[i]
        ER = xor(ER, KR)

        # S-boxes
        SR = ''
        for j in range(8):
            row = int(ER[j * 6] + ER[j * 6 + 5], 2)
            col = int(ER[j * 6 + 1:j * 6 + 5], 2)
            SR += bin(S[j][row][col])[2:].zfill(4)

        # Permutation
        SR = permute(SR, P)

        # XOR with left half
        LR = xor(L, SR)

        # Swap left and right halves
        L = R
        R = LR

    # Join left and right halves and perform final permutation
    block = permute(R + L, IP_INV)

    return block

def permute(block, table):
    """Permutes the bits in the given block according to the given table."""
    return ''.join(block[i - 1] for i in table)

def xor(block1, block2):
    """Performs the XOR operation on the two given blocks."""
    return ''.join(str(int(b1) ^ int(b2)) for b1, b2 in zip(block1, block2))

# Initial permutation

IP = [ 58, 50, 42, 34, 26, 18, 10, 2,
         60, 52, 44, 36, 28, 20, 12, 4,
         62, 54, 46, 38, 30, 22, 14, 6,
         64, 56, 48, 40, 32, 24, 16, 8,
         57, 49, 41, 33, 25, 17, 9, 1,
         59, 51, 43, 35, 27, 19, 11, 3,
         61, 53, 45, 37, 29, 21, 13, 5,
         63, 55, 47, 39, 31, 23, 15, 7 ]

# Final permutation

IP_INV = [ 40, 8, 48, 16, 56, 24, 64, 32,
                39, 7, 47, 15, 55, 23, 63, 31,
                38, 6, 46, 14, 54, 22, 62, 30,
                37, 5, 45, 13, 53, 21, 61, 29,
                36, 4, 44, 12, 52, 20, 60, 28,
                35, 3, 43, 11, 51, 19, 59, 27,
                34, 2, 42, 10, 50, 18, 58, 26,
                33, 1, 41, 9, 49, 17, 57, 25 ]

# Expansion permutation

E = [ 32, 1, 2, 3, 4, 5,
        4, 5, 6, 7, 8, 9,
        8, 9, 10, 11, 12, 13,
        12, 13, 14, 15, 16, 17,
        16, 17, 18, 19, 20, 21,
        20, 21, 22, 23, 24, 25,
        24, 25, 26, 27, 28, 29,
        28, 29, 30, 31, 32, 1 ]

# Permutation

P = [ 16, 7, 20, 21, 29, 12, 28, 17,
        1, 15, 23, 26, 5, 18, 31, 10,
        2, 8, 24, 14, 32, 27, 3, 9,
        19, 13, 30, 6, 22, 11, 4, 25 ]

# S-boxes

S = [ [ 14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7 ],
      [ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8 ],
      [ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0 ],
      [ 15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ], 
      [ 15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10 ],
      [ 3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5 ],
      [ 0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15 ],
      [ 13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ]]

# Key schedule

PC1 = [ 57, 49, 41, 33, 25, 17, 9,
            1, 58, 50, 42, 34, 26, 18,
            10, 2, 59, 51, 43, 35, 27,
            19, 11, 3, 60, 52, 44, 36,
            63, 55, 47, 39, 31, 23, 15,
            7, 62, 54, 46, 38, 30, 22,
            14, 6, 61, 53, 45, 37, 29,
            21, 13, 5, 28, 20, 12, 4 ]

PC2 = [ 14, 17, 11, 24, 1, 5,
            3, 28, 15, 6, 21, 10,
            23, 19, 12, 4, 26, 8,
            16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55,
            30, 40, 51, 45, 33, 48,
            44, 49, 39, 56, 34, 53,
            46, 42, 50, 36, 29, 32 ]

SHIFTS = [ 1, 1, 2, 2,
                2, 2, 2, 2,
                1, 2, 2, 2,
                2, 2, 2, 1 ]

def key_schedule(key):
    """Generates the round keys for the given key."""
    # Apply PC1 permutation
    key = permute(key, PC1)

    # Split into left and right halves
    C = key[:28]
    D = key[28:]

    # Generate 16 round keys
    round_keys = []
    for i in range(16):
        # Apply left shift
        C = C[SHIFTS[i]:] + C[:SHIFTS[i]]
        D = D[SHIFTS[i]:] + D[:SHIFTS[i]]

        # Join halves and apply PC2 permutation
        round_key = permute(C + D, PC2)
        round_keys.append(round_key)

    return round_keys

def generate_key():
    """Generates a random 64-bit key."""
    key = ''
    for i in range(64):
        key += str(random.randint(0, 1))
    return key

# Path: DES.ipynb
# 
# # DES
#   - Data Encryption Standard
#  - Block cipher
# - 64-bit block size
# - 56-bit key size
# - 16 rounds
# - 8 S-boxes
# - 1 initial permutation
# - 1 final permutation
# - 1 expansion permutation
# - 1 permutation
# - 1 key schedule
# 
# ## Key Schedule
# - 56-bit key
# - Apply PC1 permutation
# - Split into left and right halves
# - Generate 16 round keys
#  - Apply left shift
# - Join halves and apply PC2 permutation
# 
# ## Encryption
# - 64-bit plaintext
# - Apply initial permutation
# - Split into left and right halves
# - Perform 16 rounds
# - Expand right half
# - XOR with round key
# - S-boxes
# - Permutation
# - XOR with left half
# - Swap left and right halves
# - Join left and right halves and apply final permutation
# 
# ## Decryption
# - 64-bit ciphertext
# - Apply initial permutation
# - Split into left and right halves
# - Perform 16 rounds
# - Expand right half
# - XOR with round key
# - S-boxes
# - Permutation
# - XOR with left half
# - Swap left and right halves
# - Join left and right halves and apply final permutation
# 

plaintext = '0123456789ABCDEF'
key = '133457799BBCDFF1'

print('Plaintext:', plaintext)
print('Key:', key)

round_keys = key_schedule(key)
print('Round keys:', round_keys)

ciphertext = encrypt(plaintext, round_keys)
print('Ciphertext:', ciphertext)

plaintext = decrypt(ciphertext, round_keys)
print('Plaintext:', plaintext)


Plaintext: 0123456789ABCDEF
Key: 133457799BBCDFF1


IndexError: string index out of range

# <p style="text-align:center">Author: Nguyen Quoc Khanh </p>